# 06 - Temporal Analysis: Entropy Evolution Over Time

Analyze how project entropy and governance characteristics change over the project lifecycle.

**Goals:**
1. Track entropy evolution over commit history
2. Identify governance transition points
3. Analyze contributor churn and its effects
4. Predict sustainability based on temporal patterns

## Setup

In [ ]:
import json
import sys
from pathlib import Path
from datetime import datetime, timedelta
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats
from scipy.signal import savgol_filter

# Add src to path
sys.path.insert(0, '../src')
from analysis.entropy_calculation import EntropyCalculator

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 8]
plt.rcParams['font.size'] = 11

# Initialize entropy calculator
entropy_calc = EntropyCalculator()

print("✅ Setup complete!")

## 1. Load Project Data

In [ ]:
# Load all collected project data
data_dir = Path("../data/raw")
projects = {}

for file_path in data_dir.glob("*_data.json"):
    with open(file_path, 'r') as f:
        data = json.load(f)
        repo_name = data['repository']['full_name']
        projects[repo_name] = data

print(f"Loaded {len(projects)} projects:")
for name, data in projects.items():
    commits = len(data.get('recent_commits', []))
    print(f"  - {name}: {commits} commits")

## 2. Temporal Entropy Calculation

Calculate entropy using sliding windows over commit history.

In [ ]:
def parse_commit_date(date_str: str) -> datetime:
    """Parse commit date string to datetime."""
    try:
        return datetime.fromisoformat(date_str.replace('Z', '+00:00'))
    except:
        return None

def calculate_window_entropy(commits: list, window_days: int = 30) -> list:
    """
    Calculate entropy using sliding windows.
    
    Args:
        commits: List of commit dictionaries with 'date' and 'author'
        window_days: Size of sliding window in days
    
    Returns:
        List of (date, entropy, commit_count, unique_authors) tuples
    """
    if not commits:
        return []
    
    # Parse dates and filter valid commits
    dated_commits = []
    for c in commits:
        date = parse_commit_date(c.get('date', ''))
        if date and c.get('author'):
            dated_commits.append({'date': date, 'author': c['author']})
    
    if not dated_commits:
        return []
    
    # Sort by date
    dated_commits.sort(key=lambda x: x['date'])
    
    # Get date range
    start_date = dated_commits[0]['date']
    end_date = dated_commits[-1]['date']
    
    # Calculate entropy for each window
    results = []
    window = timedelta(days=window_days)
    step = timedelta(days=window_days // 4)  # 25% overlap
    
    current_start = start_date
    while current_start < end_date:
        current_end = current_start + window
        
        # Get commits in window
        window_commits = [
            c for c in dated_commits 
            if current_start <= c['date'] < current_end
        ]
        
        if len(window_commits) >= 5:  # Minimum commits for meaningful entropy
            # Count contributions per author
            author_counts = defaultdict(int)
            for c in window_commits:
                author_counts[c['author']] += 1
            
            # Calculate entropy
            counts = list(author_counts.values())
            total = sum(counts)
            probs = np.array(counts) / total
            entropy = -np.sum(probs * np.log2(probs + 1e-10))
            
            # Normalize by max possible entropy
            max_entropy = np.log2(len(counts))
            norm_entropy = entropy / max_entropy if max_entropy > 0 else 0
            
            # Top contributor dominance
            top_dominance = max(counts) / total * 100
            
            results.append({
                'date': current_start + window / 2,  # Window midpoint
                'entropy': entropy,
                'normalized_entropy': norm_entropy,
                'commit_count': len(window_commits),
                'unique_authors': len(author_counts),
                'top_dominance': top_dominance
            })
        
        current_start += step
    
    return results

print("✅ Temporal entropy functions defined")

## 3. Calculate Temporal Metrics

In [ ]:
# Calculate temporal entropy for all projects
temporal_data = {}

for repo_name, data in projects.items():
    commits = data.get('recent_commits', [])
    
    if len(commits) < 20:
        print(f"  ⚠️  {repo_name}: Insufficient commits ({len(commits)})")
        continue
    
    # Calculate with 30-day windows
    entropy_series = calculate_window_entropy(commits, window_days=30)
    
    if entropy_series:
        temporal_data[repo_name] = pd.DataFrame(entropy_series)
        print(f"  ✓ {repo_name}: {len(entropy_series)} data points")
    else:
        print(f"  ⚠️  {repo_name}: Could not calculate entropy series")

print(f"\n✅ Calculated temporal entropy for {len(temporal_data)} projects")

## 4. Entropy Evolution Visualization

In [ ]:
def plot_entropy_evolution(df: pd.DataFrame, repo_name: str, ax=None):
    """
    Plot entropy evolution over time with trend analysis.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 6))
    
    # Convert dates
    dates = pd.to_datetime(df['date'])
    
    # Plot normalized entropy
    ax.plot(dates, df['normalized_entropy'], 'b-', alpha=0.5, linewidth=1, label='Raw')
    
    # Smoothed line
    if len(df) >= 5:
        window = min(5, len(df) // 2)
        if window >= 3:
            smoothed = savgol_filter(df['normalized_entropy'], window, 2)
            ax.plot(dates, smoothed, 'b-', linewidth=2, label='Smoothed')
    
    # Linear trend
    x_numeric = np.arange(len(df))
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        x_numeric, df['normalized_entropy']
    )
    trend_line = slope * x_numeric + intercept
    ax.plot(dates, trend_line, 'r--', linewidth=1.5, 
           label=f'Trend (slope={slope:.4f}, R²={r_value**2:.3f})')
    
    # Formatting
    ax.set_xlabel('Date')
    ax.set_ylabel('Normalized Entropy')
    ax.set_title(f'{repo_name} - Entropy Evolution', fontsize=12, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.legend(loc='upper right')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Add interpretation
    if slope > 0.01:
        interpretation = "↑ Increasing distribution"
        color = 'green'
    elif slope < -0.01:
        interpretation = "↓ Increasing concentration"
        color = 'red'
    else:
        interpretation = "→ Stable governance"
        color = 'blue'
    
    ax.text(0.02, 0.95, interpretation, transform=ax.transAxes, 
           fontsize=10, color=color, fontweight='bold', va='top')
    
    return ax

# Plot for all projects
if temporal_data:
    n_projects = len(temporal_data)
    fig, axes = plt.subplots(n_projects, 1, figsize=(14, 5 * n_projects))
    
    if n_projects == 1:
        axes = [axes]
    
    for ax, (repo_name, df) in zip(axes, temporal_data.items()):
        plot_entropy_evolution(df, repo_name, ax)
    
    plt.tight_layout()
    plt.savefig('../docs/diagrams/temporal_entropy_evolution.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✅ Saved temporal_entropy_evolution.png")
else:
    print("No temporal data to plot")

## 5. Contributor Activity Patterns

In [ ]:
def analyze_contributor_activity(commits: list) -> pd.DataFrame:
    """
    Analyze temporal patterns of contributor activity.
    """
    if not commits:
        return pd.DataFrame()
    
    # Parse commits
    commit_data = []
    for c in commits:
        date = parse_commit_date(c.get('date', ''))
        if date and c.get('author'):
            commit_data.append({
                'date': date,
                'author': c['author'],
                'additions': c.get('additions', 0),
                'deletions': c.get('deletions', 0)
            })
    
    if not commit_data:
        return pd.DataFrame()
    
    df = pd.DataFrame(commit_data)
    df['month'] = df['date'].dt.to_period('M')
    
    # Calculate monthly statistics
    monthly = df.groupby('month').agg({
        'author': ['count', 'nunique'],
        'additions': 'sum',
        'deletions': 'sum'
    })
    monthly.columns = ['commits', 'unique_authors', 'additions', 'deletions']
    monthly['churn'] = monthly['additions'] + monthly['deletions']
    
    return monthly.reset_index()

# Analyze contributor activity
print("Contributor Activity Analysis:")
print("=" * 60)

for repo_name, data in projects.items():
    commits = data.get('recent_commits', [])
    monthly = analyze_contributor_activity(commits)
    
    if not monthly.empty:
        print(f"\n{repo_name}:")
        print(f"  Avg commits/month: {monthly['commits'].mean():.1f}")
        print(f"  Avg unique authors/month: {monthly['unique_authors'].mean():.1f}")
        print(f"  Max authors in a month: {monthly['unique_authors'].max()}")
        print(f"  Total code churn: {monthly['churn'].sum():,} lines")

## 6. Governance Transition Detection

In [ ]:
def detect_transitions(df: pd.DataFrame, threshold: float = 0.15) -> list:
    """
    Detect significant changes in entropy (potential governance transitions).
    
    Args:
        df: DataFrame with normalized_entropy column
        threshold: Minimum change to consider significant
    
    Returns:
        List of transition points with details
    """
    if len(df) < 3:
        return []
    
    transitions = []
    entropy = df['normalized_entropy'].values
    dates = pd.to_datetime(df['date'])
    
    # Calculate rolling differences
    for i in range(2, len(entropy)):
        # Compare to previous period
        prev_avg = np.mean(entropy[max(0, i-3):i])
        curr_avg = np.mean(entropy[i:min(len(entropy), i+3)])
        change = curr_avg - prev_avg
        
        if abs(change) >= threshold:
            transitions.append({
                'date': dates.iloc[i],
                'change': change,
                'direction': 'increasing' if change > 0 else 'decreasing',
                'before': prev_avg,
                'after': curr_avg,
                'interpretation': (
                    'Distribution increasing (toward Federation)' if change > 0
                    else 'Concentration increasing (toward Stadium)'
                )
            })
    
    return transitions

# Detect transitions for all projects
print("\nGovernance Transition Detection:")
print("=" * 60)

for repo_name, df in temporal_data.items():
    transitions = detect_transitions(df)
    
    print(f"\n{repo_name}:")
    if transitions:
        for t in transitions:
            print(f"  📌 {t['date'].strftime('%Y-%m')}: {t['interpretation']}")
            print(f"      Change: {t['before']:.3f} → {t['after']:.3f} ({t['change']:+.3f})")
    else:
        print(f"  No significant transitions detected")

## 7. Bus Factor Temporal Analysis

In [ ]:
def calculate_bus_factor_over_time(commits: list, window_days: int = 90) -> list:
    """
    Calculate bus factor (minimum contributors for 50% of commits) over time.
    """
    if not commits:
        return []
    
    # Parse commits
    dated_commits = []
    for c in commits:
        date = parse_commit_date(c.get('date', ''))
        if date and c.get('author'):
            dated_commits.append({'date': date, 'author': c['author']})
    
    if not dated_commits:
        return []
    
    dated_commits.sort(key=lambda x: x['date'])
    
    results = []
    window = timedelta(days=window_days)
    step = timedelta(days=window_days // 3)
    
    start_date = dated_commits[0]['date']
    end_date = dated_commits[-1]['date']
    
    current = start_date
    while current < end_date:
        window_end = current + window
        window_commits = [c for c in dated_commits if current <= c['date'] < window_end]
        
        if len(window_commits) >= 10:
            # Count contributions
            author_counts = defaultdict(int)
            for c in window_commits:
                author_counts[c['author']] += 1
            
            # Sort by contributions
            sorted_authors = sorted(author_counts.values(), reverse=True)
            total = sum(sorted_authors)
            
            # Calculate bus factor (contributors for 50%)
            cumsum = 0
            bus_factor = 0
            for count in sorted_authors:
                cumsum += count
                bus_factor += 1
                if cumsum >= total * 0.5:
                    break
            
            results.append({
                'date': current + window / 2,
                'bus_factor': bus_factor,
                'total_contributors': len(author_counts),
                'top_contributor_pct': sorted_authors[0] / total * 100
            })
        
        current += step
    
    return results

# Calculate bus factor over time
bus_factor_data = {}

for repo_name, data in projects.items():
    commits = data.get('recent_commits', [])
    bf_series = calculate_bus_factor_over_time(commits)
    
    if bf_series:
        bus_factor_data[repo_name] = pd.DataFrame(bf_series)

# Plot bus factor evolution
if bus_factor_data:
    fig, axes = plt.subplots(len(bus_factor_data), 1, figsize=(12, 4 * len(bus_factor_data)))
    
    if len(bus_factor_data) == 1:
        axes = [axes]
    
    for ax, (repo_name, df) in zip(axes, bus_factor_data.items()):
        dates = pd.to_datetime(df['date'])
        
        ax.plot(dates, df['bus_factor'], 'r-', linewidth=2, marker='o', markersize=4)
        ax.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Critical (1)')
        ax.axhline(y=3, color='orange', linestyle='--', alpha=0.5, label='Warning (3)')
        
        ax.set_xlabel('Date')
        ax.set_ylabel('Bus Factor')
        ax.set_title(f'{repo_name} - Bus Factor Over Time', fontweight='bold')
        ax.set_ylim(0, max(10, df['bus_factor'].max() + 1))
        ax.legend()
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    
    plt.tight_layout()
    plt.savefig('../docs/diagrams/bus_factor_temporal.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✅ Saved bus_factor_temporal.png")

## 8. Summary Statistics

In [ ]:
# Generate temporal summary
summary_data = []

for repo_name in temporal_data:
    df = temporal_data[repo_name]
    
    # Trend analysis
    x = np.arange(len(df))
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        x, df['normalized_entropy']
    )
    
    # Bus factor stats
    bf_df = bus_factor_data.get(repo_name)
    avg_bus_factor = bf_df['bus_factor'].mean() if bf_df is not None else None
    
    summary_data.append({
        'repository': repo_name,
        'mean_entropy': df['normalized_entropy'].mean(),
        'std_entropy': df['normalized_entropy'].std(),
        'trend_slope': slope,
        'trend_r2': r_value ** 2,
        'trend_significant': p_value < 0.05,
        'avg_bus_factor': avg_bus_factor,
        'trend_direction': 'Federating' if slope > 0.001 else ('Concentrating' if slope < -0.001 else 'Stable')
    })

df_summary = pd.DataFrame(summary_data)

print("\n" + "=" * 70)
print("TEMPORAL ANALYSIS SUMMARY")
print("=" * 70)
display(df_summary.round(4))

# Save results
output_path = Path('../data/processed/temporal_analysis.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
df_summary.to_csv(output_path, index=False)
print(f"\n✅ Summary saved to {output_path}")

## Key Findings

**Interpretation:**
- **Positive slope**: Project is "federating" - contributions becoming more distributed
- **Negative slope**: Project is "concentrating" - moving toward Stadium pattern
- **Low bus factor + negative slope**: High sustainability risk
- **Significant transitions**: May indicate key contributor departures or onboarding

**Next Steps:**
1. Correlate transitions with external events (releases, security issues)
2. Compare temporal patterns across project categories
3. Build predictive model for sustainability risk